## 基于item的协同过滤

基本思想:
* 如果⽤户喜欢物品𝑖𝑡𝑒𝑚_1，⽽且物品𝑖𝑡𝑒𝑚_1 与𝑖𝑡𝑒𝑚_2 相似，那么⽤户很可能喜欢物品𝑖𝑡𝑒𝑚_2。

In [1]:
%cd /playground/sgd_deep_learning/sgd_rec_sys/
import sys 
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rec_sys


In [2]:
import numpy as np
import random
from sgd_rec_sys.retrieval import ItemCF, RateInfo

## rate_info

* 从文件中读取用户、物品的meta info（比如id-name的映射关系）
* 读取用户历史评分文件，针对不同算法整理对应数据
  * itemcf：需要每个物品 对应的 用户评价list
  * usercf：需要每个用户 评价过的 所有物品的list

In [3]:
rate_info = RateInfo(user_file='./data/retrieval/user2id.txt',
                     item_file='./data/retrieval/item2id.txt',
                    rate_file='./data/retrieval/userid_itemid_rate.txt')

In [4]:
# 用户侧信息
rate_info.user_meta_info()

{'col_name': ['user_id', 'user_name'],
 'id2name': {1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E'},
 'name2id': {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}}

In [5]:
# 物品侧信息
rate_info.item_meta_info()

{'col_name': ['item_id', 'item_name'],
 'id2name': {1: 'story_book', 2: 'magazine', 3: 'tv', 4: 'ps4'},
 'name2id': {'story_book': 1, 'magazine': 2, 'tv': 3, 'ps4': 4}}

In [6]:
#  rate 信息
rate_info.rate_meta_info()

{'col_name': ['userid', 'itemid', 'rate'],
 'rate_pairs': [[1, 1, 1],
  [1, 2, -1],
  [1, 3, 1],
  [1, 4, 1],
  [2, 2, 1],
  [2, 3, -1],
  [2, 4, -1],
  [3, 1, 1],
  [3, 2, 1],
  [3, 3, -1],
  [4, 1, -1],
  [4, 3, 1],
  [5, 1, 1],
  [5, 2, 1],
  [5, 4, -1]]}

## ItemCF

In [10]:
itemcf = ItemCF(meta_info=rate_info)

item_info = rate_info.item_meta_info()
iids = list(item_info['id2name'].keys())
print("all item ids:", iids)
print(item_info['id2name'])
print()

# 计算两两物品间的cos sim (耗时操作可离线计算)
for i in range(len(iids)-1):
    for j in range(i, len(iids)):
        id1, id2 = iids[i], iids[j]
        print("sim score of {}-{} :\t {}\n".format(id1, id2, itemcf.sim(id1, id2)))
        

all item ids: [1, 2, 3, 4]
{1: 'story_book', 2: 'magazine', 3: 'tv', 4: 'ps4'}

w1, w2 {1: 1, 3: 1, 4: -1, 5: 1} {1: 1, 3: 1, 4: -1, 5: 1}
common [1, 3, 4, 5]
sim score of 1-1 :	 1.0

w1, w2 {1: 1, 3: 1, 4: -1, 5: 1} {1: -1, 2: 1, 3: 1, 5: 1}
common [1, 3, 5]
sim score of 1-2 :	 0.25

w1, w2 {1: 1, 3: 1, 4: -1, 5: 1} {1: 1, 2: -1, 3: -1, 4: 1}
common [1, 3, 4]
sim score of 1-3 :	 -0.25

w1, w2 {1: 1, 3: 1, 4: -1, 5: 1} {1: 1, 2: -1, 5: -1}
common [1, 5]
sim score of 1-4 :	 0.0

w1, w2 {1: -1, 2: 1, 3: 1, 5: 1} {1: -1, 2: 1, 3: 1, 5: 1}
common [1, 2, 3, 5]
sim score of 2-2 :	 1.0

w1, w2 {1: -1, 2: 1, 3: 1, 5: 1} {1: 1, 2: -1, 3: -1, 4: 1}
common [1, 2, 3]
sim score of 2-3 :	 -0.75

w1, w2 {1: -1, 2: 1, 3: 1, 5: 1} {1: 1, 2: -1, 5: -1}
common [1, 2, 5]
sim score of 2-4 :	 -0.8660254037844387

w1, w2 {1: 1, 2: -1, 3: -1, 4: 1} {1: 1, 2: -1, 3: -1, 4: 1}
common [1, 2, 3, 4]
sim score of 3-3 :	 1.0

w1, w2 {1: 1, 2: -1, 3: -1, 4: 1} {1: 1, 2: -1, 5: -1}
common [1, 2]
sim score of 3-4 :	 0.

## ItemCF召回的完整流程

建⽴ ⽤户->物品 的索引
* 记录每个⽤户最近点击、交互过的物品ID。
* 给定任意⽤户ID，可以找到他近期感兴趣的物品列表。
  
建⽴ 物品->物品 的索引
* 计算物品之间两两相似度。
* 对于每个物品，索引它最相似的k个物品。
* 给定任意物品ID，可以快速找到它最相似的k个物品。

线上做召回
1) 给定⽤户ID，通过⽤户->物品索引，找到⽤户近期感兴趣的物品列表（last-n）。
2) 对于last-n列表中每个物品，通过物品->物品的索引，找到top-k相似物品。
3) 对于取回的相似物品（最多有𝑛𝑘 个），⽤公式预估⽤户对物品的兴趣分数。
4) 返回分数最⾼的100个物品，作为推荐结果。

⽤索引，离线计算量⼤，线上计算量⼩。

